
<center><h1> Statistics of Indian School Education </h1></center>

<center>The goal of this notebook is to dig some facts about the quality of Indian Education from 2013 - 2016 and help people of India and Government of India to understand better about the state of Indian Schools to take immediate actions if needed.</center>

<div style="text-align:center"><img src="https://www.shethepeople.tv/wp-content/uploads/2017/09/RIGHT-TO-EDUCATION.jpg"/></div>

# Table Of Contents

1. Exploratory Data Analysis (EDA)
    - What do you gain through EDA?
    - EDA Techniques
2. Gross Enrollment Ratio (GER)
    - Calculation Method
    - All India GER
    - State-wise GER
3. Dropout Ratio
    - All India Dropout Ratio
    - State-wise Dropout Ratio
4. Percentage of Schools with Electricity, Water, Toilet and Computer Facilities
5. Conclusion

# Exploratory Data Analysis (EDA)

Although many of you might be familiar with what EDA is all about, I would like to take a minute and give a formal definition of EDA and set the tone for this notebook for beginners and experts as well.

In statistics, exploratory data analysis is an approach to analyzing data sets to summarize their main characteristics, often with visual methods. A statistical model can be used or not, but primarily EDA is for seeing what the data can tell us beyond the formal modeling or hypothesis testing task. Exploratory data analysis was promoted by **John Tukey** to encourage statisticians to explore the data, and possibly formulate hypotheses that could lead to new data collection and experiments.


## What do you gain through EDA?

Below are few among many points where EDA helps us:

- maximize insight into a data set;
- uncover underlying structure;
- extract important variables;
- detect outliers and anomalies;
- test underlying assumptions;
- develop parsimonious models; and
- determine optimal factor settings.

## EDA Techniques

Most EDA techniques are graphical in nature with a few quantitative techniques. The reason for the heavy reliance on graphics is that by its very nature the main role of EDA is to open-mindedly explore, and graphics gives the analysts unparalleled power to do so, enticing the data to reveal its structural secrets, and being always ready to gain some new, often unsuspected, insight into the data.

The particular graphical techniques employed in EDA are often quite simple, consisting of various techniques of:

- Plotting the raw data (such as data traces, histograms, bihistograms, probability plots, lag plots, block plots, and Youden plots.
- Plotting simple statistics such as mean plots, standard deviation plots, box plots, and main effects plots of the raw data.
- Positioning such plots so as to maximize our natural pattern-recognition abilities, such as using multiple plots per page.

# Gross Enrollment Ratio

Gross Enrolment Ratio (GER) or Gross Enrolment Index (GEI) is a statistical measure used in the education sector, to determine the **number of students enrolled in school at several different grade levels (like elementary, middle school and high school), and use it to show the ratio of the number of students who live in that country to those who qualify for the particular grade level**.

The GER can be over 100% as it includes students who may be older or younger than the official age group. 

For instance, in India it improved from 25.8 to 26.3, the **GER includes students who are repeating a grade, those who enrolled late and are older than their classmates, or those who have advanced quickly and are younger than their classmates**. This allows the total enrolment to exceed the population that corresponds to that level of education.

## Calculation Method

### a = number of students enrolled in a given level
### b = population of the age group corresponds to given level of education India

### $GER = \frac{a}{b} \times 100$

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
import descartes

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_ger = pd.read_csv('../input/indian-school-education-statistics/gross-enrollment-ratio-2013-2016.csv')
df_ger.head(10)

In [ ]:
df_ger.info()

In [ ]:
# Arrange the `year` column in ascending order
df_ger = df_ger.sort_values('Year',ascending=True)

# Dataframe for `All India` Gross Enrollment Ratio
ger_ai = df_ger[df_ger['State_UT'] == 'All India'].reset_index(drop=True)

In [ ]:
print("Gross Enrollment Ratio - All India (2013-2016)")
display(ger_ai)

As you can see, the columns `Higher_Secondary_Boys`,`Higher_Secondary_Girls`, and `Higher_Secondary_Total` has `dtype` as `object` instead of `float` like the rest of them. When I tried to change the column into `float` type, I got the error that there's a string `NR` inside one or more of the cells. Which means that we have few rows where the data was **NOT RECORDED**. This doesn't necessarily mean we should substitute that value with zero because that undermines the total Gross Enrollment Ratio (refer to the calculation method of GER). Instead, we will just drop the rows which contain these strings or other dtypes if it's not a float.

**_TL;DR_** 
- **Removing `NR` and `@` strings which are in the columns: Higher_Secondary_Boys`,`Higher_Secondary_Girls`, and `Higher_Secondary_Total`**

In [ ]:
def drop_rows(dataframe,given_list,search_value):
    """Dropping the rows which are not `float64` type.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
    
    given_list : list
                List of column names
    
    search_value : string
                String to search and remove the corresponding row
    
    Returns
    -------
    dataframe : pandas.DataFrame
    """
    for i in given_list:
        dataframe = dataframe[~dataframe[i].str.contains(search_value)].reset_index(drop=True)
    
    return dataframe

In [ ]:
ger_column_list = ['Higher_Secondary_Boys','Higher_Secondary_Girls','Higher_Secondary_Total']

In [ ]:
df_ger = drop_rows(df_ger,ger_column_list,'NR')
df_ger = drop_rows(df_ger,ger_column_list,'@')

Now that we presumably removed all eccentric rows, let's convert these columns into `float` type so that we can work with the dataframe.

In [ ]:
def convert_to_float(dataframe,given_list,which_type):
    """To convert a column to desired type. In this case, from object to float.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
    
    given_list : list
                List of column names for which data type should be converted.
    
    which_type : dtype
    
    Returns
    -------
    dataframe : pandas.DataFrame
    """
    for i in given_list:
        dataframe[i] = dataframe[i].astype(which_type)
    return dataframe

In [ ]:
df_ger = convert_to_float(df_ger,ger_column_list,float)

We now have a clean dataframe to work with.

In [ ]:
df_ger.info()

Let's compare **All India Gross Enrollment Ratio** between Girls and Boys from 2013 - 2016.

In [ ]:
ger_girls_list = ['State_UT','Year','Primary_Girls','Upper_Primary_Girls','Secondary_Girls','Higher_Secondary_Girls']
ger_boys_list = ['State_UT','Year','Primary_Boys','Upper_Primary_Boys','Secondary_Boys','Higher_Secondary_Boys']

ger_girls = ger_ai.loc[:,ger_girls_list]
ger_boys = ger_ai.loc[:,ger_boys_list]

In [ ]:
# preparing the dataframe for seaborn plots

tidy_girls = pd.melt(ger_girls,id_vars=['Year','State_UT']).rename(columns=str.title)
tidy_boys = pd.melt(ger_boys,id_vars=['Year','State_UT']).rename(columns=str.title)

## All India GER

In [ ]:
# setting the plot styles and backgrounds
sns.set(style="ticks",context="talk")
plt.style.use('dark_background')

# creating two subplots: girls and boys
f, axes = plt.subplots(1, 2, figsize=(20, 10))

# -------------------------------------------------------------- #

# barplot for girls
ger_girls_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_girls,edgecolor='black',ax=axes[0])
ger_girls_bplot.set(ylim=(40, 130))
for p in ger_girls_bplot.patches:
    ger_girls_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes[0].set_title('Gross Enrollment Ratio of Girls in India',size = 16 , pad = 16, color='black',bbox=dict(facecolor='white', alpha=1.0))

axes[0].set_ylabel('Gross Enrollment Ratio',size=13)
axes[0].set_xlabel('Year',size=13)

ger_girls_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

# ------------------------------------------------------------------- #

# barplot for boys
ger_boys_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_boys,edgecolor='black',ax=axes[1])
ger_boys_bplot.set(ylim=(40, 130))
for p in ger_boys_bplot.patches:
    ger_boys_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes[1].set_title('Gross Enrollment Ratio of Boys in India',size = 16 , pad = 16, color='black',bbox=dict(facecolor='white', alpha=1.0))

axes[1].set_ylabel('Gross Enrollment Ratio',size=13)
axes[1].set_xlabel('Year',size=13)

ger_boys_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

### Girls

Let's see how the trends have been in Gross Enrollment Ratio for Girls.

- Girls who join Primary School have exceeded 100 GER which means the students also include who are repeating a grade, or those who enrolled late and are older than their classmates, or those who have advanced quickly and are younger than their classmates. This is a positive sign in a developing country like India.
- However, the ratio has been consistently decreasing over the years. There could be two reasons for this decrease:
    1. Either students who are repeating grades & those who have enrolled late are decreasing over time,
    2. Or, there's an actual decrease in enrollments for primary schools among girls.
- Indian Government has look into this to take a quick action if it's the second reason.
- There's a considerable increase in GER from 2013-2016 for girls who are joining Upper Primary, Secondary, and Higher Secondary Schools. This is again a very positive sign.

### Boys

Let's see how the trends have been in Gross Enrollment Ratio for Boys.

- The trends for Primary Schools are the same as girls. Again, if we could look further into data about the GER, we could gather some important information as to why there's a decrease in GER for Girls and Boys.
- Compared to Girls, the GER of Boys in Upper Primary is very low. This is a negative sign because this might affect the enrollments of Secondary and Higher Secondary Schools among boys.

### General

- We can also observe that there's a big difference between Secondary and Higher Secondary Enrollments. There could be many reasons like:
    1. Students might migrate to better colleges in cities/towns.
    2. They might enroll themselves in Coaching Centres such as IIT/AIEEE/etc.


## State-wise GER

Let's visualize the Gross Enrollment Ratio of all the states in India using an India Map.

In [ ]:
# Dataframe for only states

ger_states = df_ger[df_ger['State_UT'] != 'All India'].reset_index(drop=True)

In [ ]:
ger_states.State_UT.unique()

In [ ]:
#renaming state names
ger_states['State_UT'].replace({"Andaman & Nicobar Islands": "A and N Islands",
                      'Jammu And Kashmir':'J & K',
                      'MADHYA PRADESH':'Madhya Pradesh',
                      'Pondicherry':'Puducherry'
                     }, inplace=True)

ger_states_girls = ger_states.loc[:,ger_girls_list]
ger_states_boys = ger_states.loc[:,ger_boys_list]

In [ ]:
# Add 'india-states' dataset to your working directory and perform the analysis

fp = "../input/india-states/Igismap/Indian_States.shp"
map_df = gpd.read_file(fp)

# renaming state names
map_df['st_nm'].replace({"Andaman & Nicobar Island": "A and N Islands",
                      "Arunanchal Pradesh": "Arunachal Pradesh",
                      'Dadara & Nagar Havelli':'Dadra & Nagar Haveli',
                      'Jammu & Kashmir':'J & K',
                      'NCT of Delhi':'Delhi',
                     }, inplace=True)

# Merging `ger_states_girls` and `map_df`

merged_girls = map_df.set_index('st_nm').join(ger_states_girls.set_index('State_UT')).dropna()
merged_boys = map_df.set_index('st_nm').join(ger_states_boys.set_index('State_UT')).dropna()

# use this if you want to annotate the map with numbers

# merged_girls['coords'] = merged_girls['geometry'].apply(lambda x: x.representative_point().coords[:])
# merged_girls['coords'] = [coords[0] for coords in merged_girls['coords']]

In [ ]:
# State-wise plots of GER for Girls in different grade levels

fig, ax = plt.subplots(2, 2, figsize=(30, 20))
ax[0,0].axis('off')
ax[0,1].axis('off')
ax[1,0].axis('off')
ax[1,1].axis('off')

# some variables which you can play with

fontdict = {'fontsize': '15', 'fontweight' : '3'}
pad = 16
color = 'black'
bbox = dict(facecolor='white', alpha=1.0)

cmap = 'inferno'
linewidth = 0.8
edgecolor = 'white'

ax[0,0].set_title("State-Wise GER of Primary_Girls (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

g1 = merged_girls.plot(column='Primary_Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

ax[0,1].set_title("State-Wise GER of Upper_Primary_Girls (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

g2 = merged_girls.plot(column='Upper_Primary_Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

ax[1,0].set_title("State-Wise GER of Secondary_Girls (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)
g3 = merged_girls.plot(column='Secondary_Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

ax[1,1].set_title("State-Wise GER of Higher_Secondary_Girls (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

g4 = merged_girls.plot(column='Higher_Secondary_Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

### Girls

- It's clear from the graph that the Primary_Girls GER is the highest for Eastern states like Assam, Mizoram, etc and it's the lowest for Andhra Pradesh and J & K.
- Upper_Primary_Girls GER is the lowest in J & K and that too at an alarming number! AP also has a lot of room for improvement in its GER.
- Secondary_Girls GER top 3 lowest states are J & K, Uttar Pradesh, and Gujarat. This is really concerning. 
- I see a lot of Yellow and Orange in different states for Higher_Secondary_Girls and that's a really good sign!

In [ ]:
# State-wise plots of GER for Boys in different grade levels

fig, ax = plt.subplots(2, 2, figsize=(30, 20))
ax[0,0].axis('off')
ax[0,1].axis('off')
ax[1,0].axis('off')
ax[1,1].axis('off')

fontdict = {'fontsize': '15', 'fontweight' : '3'}
pad = 16
color = 'black'
bbox = dict(facecolor='white', alpha=1.0)

cmap = 'gist_heat'
linewidth = 0.8
edgecolor = 'white'

ax[0,0].set_title("State-Wise GER of Primary_Boys (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

b1 = merged_boys.plot(column='Primary_Boys',cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

ax[0,1].set_title("State-Wise GER of Upper_Primary_Boys (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

b2 = merged_boys.plot(column='Upper_Primary_Boys', cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

ax[1,0].set_title("State-Wise GER of Secondary_Boys (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

b3 = merged_boys.plot(column='Secondary_Boys', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

ax[1,1].set_title("State-Wise GER of Higher_Secondary_Boys (2013 - 2016)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

b4 = merged_boys.plot(column='Higher_Secondary_Boys', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "GER"})

### Boys

- Like Girls, Primary_Boys' GER also follows almost similar pattern.
- Top 2 Lowest for Upper_Primary_Boys' GER are J & K and Uttar Pradesh.
- Compared to Girls, Boys' GER in Secondary Schools is not so alarming.
- The pattern for Higher Secondary Boys look similar to the Girls.

## Note

- **These maps look really nice and easy on the eyes to grab attention for a reader. However, it's too easy to lose important information on states which are really tiny like: Goa, Andaman and Nicobar Islands, Daman and Diu, etc.**

To overcome this problem, you can do the following:

1. Make barplots for every state and annotate them with numbers to get a better understanding.
2. Use plotly to map geographical data.
3. Plot few states separately to understand which colour they fall under.
4. Annotate the maps with the GER numbers. 

I tried the 4th method, but failed to produce legible labels, they were too crowded to look visually pleasing. So dropped this idea.

I am refraining myself from performing any of the other methods because my goal (yours can differ) is to gain a high-level understanding of how GER is performing across states. I am not really concerned about Union Territories as of now.

# Dropout Ratio 

In [ ]:
df_dr = pd.read_csv('/kaggle/input/indian-school-education-statistics/dropout-ratio-2012-2015.csv')
df_dr.head(10)

In [ ]:
df_dr.info()

Clearly we have many `NR` or values other than `float` type. Let's re-use our `drop_rows()` and `convert_to_float()` functions on this dataset as well.

In [ ]:
# Arrange the `year` column in ascending order
df_dr = df_dr.sort_values('year',ascending=True)

# Dataframe for `All India` Gross Enrollment Ratio
dr_ai = df_dr[df_dr['State_UT'] == 'All India'].reset_index(drop=True)

In [ ]:
dr_column_list = ['Primary_Boys','Primary_Girls','Primary_Total','Upper Primary_Boys',
       'Upper Primary_Girls','Upper Primary_Total','Secondary _Boys','Secondary _Girls',
       'Secondary _Total','HrSecondary_Boys','HrSecondary_Boys','HrSecondary_Girls','HrSecondary_Total']

In [ ]:
df_dr = drop_rows(df_dr,dr_column_list,'NR')
df_dr = drop_rows(df_dr,['Upper Primary_Boys'],'Uppe_r_Primary') #anomaly

In [ ]:
df_dr = convert_to_float(df_dr,dr_column_list,float)

In [ ]:
df_dr.info()

We're left with total 55 rows! That's a lot of `NR` values.

In [ ]:
print("Drop Out Ratio - All India (2012-2015)")
display(dr_ai)

Since we have **NR** values only for `HrSecondary`, I will be removing all the 3 columns related to it to perform All India Drop Out Ratio analysis like we did for GER.

In [ ]:
dr_girls_list = ['State_UT','year','Primary_Girls','Upper Primary_Girls','Secondary _Girls']
dr_boys_list = ['State_UT','year','Primary_Boys','Upper Primary_Boys','Secondary _Boys']

dr_girls = dr_ai.loc[:,dr_girls_list]
dr_boys = dr_ai.loc[:,dr_boys_list]

In [ ]:
# preparing the dataframe for seaborn plots

tidy_girls = pd.melt(dr_girls,id_vars=['year','State_UT']).rename(columns=str.title)
tidy_boys = pd.melt(dr_boys,id_vars=['year','State_UT']).rename(columns=str.title)

## All India Dropout Ratio

In [ ]:
# setting the plot styles and backgrounds
sns.set(style="ticks",context="talk")
plt.style.use('dark_background')

# creating two subplots: girls and boys
f, axes = plt.subplots(1, 2, figsize=(20, 10))

# -------------------------------------------------------------- #

# barplot for girls
dr_girls_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_girls,edgecolor='black',ax=axes[0])
dr_girls_bplot.set(ylim=(0, 20))
for p in dr_girls_bplot.patches:
    dr_girls_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes[0].set_title('Dropout Ratio of Girls in India',size = 16 , pad = 16, color='black',bbox=dict(facecolor='white', alpha=1.0))

axes[0].set_ylabel('Dropout Ratio',size=13)
axes[0].set_xlabel('Year',size=13)

dr_girls_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

# ------------------------------------------------------------------- #

# barplot for boys
dr_boys_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_boys,edgecolor='black',ax=axes[1])
dr_boys_bplot.set(ylim=(0, 20))
for p in dr_boys_bplot.patches:
    dr_boys_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes[1].set_title('Dropout Ratio of Boys in India',size = 16 , pad = 16, color='black',bbox=dict(facecolor='white', alpha=1.0))

axes[1].set_ylabel('Dropout',size=13)
axes[1].set_xlabel('Year',size=13)

dr_boys_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

**What is Dropout Ratio?**

There are varying definitions on the web for Dropout Ratio. I will keep it simple here. Dropout Ratio simply means any student who leaves school for any reason before graduation or completion of a program of studies without transferring to another school.

It is very clear from the graph that we have high number of dropouts from **Secondary Girls**. Secondary Education entails 9th and 10th grades. So, there could be many reasons why the dropout is increasing at this grade level. It could be one of these reasons:

- Students feel the pressure of 10th board exams and do not possess the will to continue education thereafter.
- Since India is a developing country, many girl students from rural areas do not have support to continue education after reaching this grade and they simply think this level of education is enough for girls.
- etc.
- The same pattern of Dropout Ratio is being repeated for Boys as well.
- This is a very serious issue that Indian Government should be considering and find the root cause for it to solve this problem.

## State-wise Dropout Ratio

If you carefully observed our `df_dr` dataframe after dropping rows, you'd notice that a lot of rows had `NR` in them. As I said it's unfair to put 0 as their value because we'd be taking 0 as our actual GER or Drop Out Ratio value. So, dropping those rows entirely seems the right way to make some head way in our analysis.

Since Dropout Ratio file has so many `NR` values, I decided that I will consider only those states which have all 3 years records for my **State-wise Dropout Ratio analysis**. This seems fair to me. Let me know your thoughts on this or if there's another work around. 

In [ ]:
dr_states = (pd.concat(g for _, g in df_dr.groupby("State_UT") if len(g) == 3)).reset_index(drop=True)

In [ ]:
dr_states.State_UT.unique()

In [ ]:
dr_girls_list = ['State_UT','year','Primary_Girls','Upper Primary_Girls','Secondary _Girls','HrSecondary_Girls']
dr_boys_list = ['State_UT','year','Primary_Boys','Upper Primary_Boys','Secondary _Boys','HrSecondary_Boys']

In [ ]:
#renaming state names
dr_states['State_UT'].replace({'Jammu & Kashmir':'J & K'}, inplace=True)

dr_states_girls = dr_states.loc[:,dr_girls_list]
dr_states_boys = dr_states.loc[:,dr_boys_list]

In [ ]:
# Merging `dr_states_girls` and `map_df`

merged_girls = map_df.set_index('st_nm').join(dr_states_girls.set_index('State_UT')).dropna()
merged_boys = map_df.set_index('st_nm').join(dr_states_boys.set_index('State_UT')).dropna()

In [ ]:
# State-wise plots of Dropout Ratio for Girls in different grade levels

fig, ax = plt.subplots(2, 2, figsize=(30, 20))
ax[0,0].axis('off')
ax[0,1].axis('off')
ax[1,0].axis('off')
ax[1,1].axis('off')

# some variables which you can play with

fontdict = {'fontsize': '15', 'fontweight' : '3'}
pad = 16
color = 'black'
bbox = dict(facecolor='white', alpha=1.0)

cmap = 'PuBuGn'
linewidth = 0.8
edgecolor = 'white'

ax[0,0].set_title("State-Wise Dropout Ratio of Primary_Girls (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

g1 = merged_girls.plot(column='Primary_Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

ax[0,1].set_title("State-Wise Dropout Ratio of Upper Primary_Girls (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

g2 = merged_girls.plot(column='Upper Primary_Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

ax[1,0].set_title("State-Wise Dropout of Secondary _Girls (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)
g3 = merged_girls.plot(column='Secondary _Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

ax[1,1].set_title("State-Wise Dropout Ratio of HrSecondary_Girls (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

g4 = merged_girls.plot(column='HrSecondary_Girls', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

### Girls

- A very low Dropout Ratio in Primary girls is a good sign.
- Gujarat failed to decrease Dropout Ratio in Upper Primary and Secondary Girls. However, Gujarat has the least dropout ratio in Higher Secondary Girls.
- We can see that the dropout ratio keeps on increasing as the grade levels are increasing.

In [ ]:
# State-wise plots of Dropout Ratio for Boys in different grade levels

fig, ax = plt.subplots(2, 2, figsize=(30, 20))
ax[0,0].axis('off')
ax[0,1].axis('off')
ax[1,0].axis('off')
ax[1,1].axis('off')

# some variables which you can play with

fontdict = {'fontsize': '15', 'fontweight' : '3'}
pad = 16
color = 'black'
bbox = dict(facecolor='white', alpha=1.0)

cmap = 'BuPu'
linewidth = 0.8
edgecolor = 'white'

ax[0,0].set_title("State-Wise Dropout Ratio of Primary_Boys (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

b1 = merged_boys.plot(column='Primary_Boys', cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

ax[0,1].set_title("State-Wise Dropout Ratio of Upper Primary_Boys (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

b2 = merged_boys.plot(column='Upper Primary_Boys', cmap=cmap, linewidth=linewidth, 
                  ax=ax[0,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

ax[1,0].set_title("State-Wise Dropout of Secondary _Boys (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)
b3 = merged_boys.plot(column='Secondary _Boys', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,0], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

ax[1,1].set_title("State-Wise Dropout Ratio of HrSecondary_Boys (2012 - 2015)", fontdict=fontdict,
                  pad = pad, color=color,bbox=bbox)

b4 = merged_boys.plot(column='HrSecondary_Boys', cmap=cmap, linewidth=linewidth, 
                  ax=ax[1,1], edgecolor=edgecolor, legend=True,legend_kwds={'label': "DR"})

### Boys

- A very low dropout ratio for Boys in both Primary and Upper Primary grades.
- Gujarat again fails to lower the dropout ratio among boys as well in Secondary grades.
- A same pattern follows for Higher Secondary Boys as Girls.

## Note

The dataset for these statistics are acquired from Indian Government's Open Data Source Portal. It's really concerning to see so much of data is Not Recorded for both GER and Dropout Ratio.

# Percentage of Schools with Electricity, Water, Toilet and Computer Facilities

In [ ]:
df_water = pd.read_csv('../input/indian-school-education-statistics/percentage-of-schools-with-water-facility-2013-2016.csv')
df_comps = pd.read_csv('../input/indian-school-education-statistics/percentage-of-schools-with-comps-2013-2016.csv')
df_elec = pd.read_csv('../input/indian-school-education-statistics/percentage-of-schools-with-electricity-2013-2016.csv')
df_g_toilet = pd.read_csv('/kaggle/input/indian-school-education-statistics/schools-with-girls-toilet-2013-2016.csv')
df_b_toilet = pd.read_csv('/kaggle/input/indian-school-education-statistics/schools-with-boys-toilet-2013-2016.csv')

In [ ]:
# Arrange the `year` column in ascending order
df_water = df_water.sort_values('Year',ascending=True)
df_comps = df_comps.sort_values('year',ascending=True)
df_elec = df_elec.sort_values('year',ascending=True)
df_g_toilet = df_g_toilet.sort_values('year',ascending=True)
df_b_toilet = df_g_toilet.sort_values('year',ascending=True)

# Dataframe for `All India` Gross Enrollment Ratio
water_ai = df_water[df_water['State/UT'] == 'All India'].reset_index(drop=True)
comps_ai = df_comps[df_comps['State_UT'] == 'All India'].reset_index(drop=True)
elec_ai = df_elec[df_elec['State_UT'] == 'All India'].reset_index(drop=True)
gt_ai = df_g_toilet[df_g_toilet['State_UT'] == 'All India'].reset_index(drop=True)
bt_ai = df_b_toilet[df_b_toilet['State_UT'] == 'All India'].reset_index(drop=True)

In [ ]:
water_columns = ['State/UT','Year','Primary_Only','U_Primary_Only','Sec_Only','HrSec_Only']
comps_columns = ['State_UT','year','Primary_Only','U_Primary_Only','Sec_Only','HrSec_Only']
elec_columns = ['State_UT','year','Primary_Only','U_Primary_Only','Sec_Only','HrSec_Only']
gbt_columns = ['State_UT','year','Primary_Only','U_Primary_Only','Sec_Only','HrSec_Only']

In [ ]:
water_ai = water_ai.loc[:,water_columns]
comps_ai = comps_ai.loc[:,comps_columns]
elec_ai = elec_ai.loc[:,elec_columns]
gt_ai = gt_ai.loc[:,gbt_columns]
bt_ai = bt_ai.loc[:,gbt_columns]

In [ ]:
print("Water Facilities - All India (2013-2016)")
display(water_ai)

In [ ]:
print("Computer Facilities - All India (2013-2016)")
display(comps_ai)

In [ ]:
print("Electricy Facilities - All India (2013-2016)")
display(elec_ai)

In [ ]:
print("Toilet Facilities for Girls - All India (2013-2016)")
display(gt_ai)

In [ ]:
print("Toilet Facilities for Boys - All India (2013-2016)")
display(bt_ai)

In [ ]:
# preparing the dataframe for seaborn plots

tidy_water = pd.melt(water_ai,id_vars=['Year','State/UT']).rename(columns=str.title)
tidy_comps = pd.melt(comps_ai,id_vars=['year','State_UT']).rename(columns=str.title)
tidy_elec = pd.melt(elec_ai,id_vars=['year','State_UT']).rename(columns=str.title)
tidy_gt = pd.melt(gt_ai,id_vars=['year','State_UT']).rename(columns=str.title)
tidy_bt = pd.melt(bt_ai,id_vars=['year','State_UT']).rename(columns=str.title)

In [ ]:
# setting the plot styles and backgrounds
sns.set(style="ticks",context="talk")
plt.style.use('seaborn-deep')

f, axes = plt.subplots(1, 1, figsize=(20, 10))

# barplot for water facility
water_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_water,palette = 'ocean',edgecolor='black',ax=axes)
water_bplot.set(ylim=(0, 150))
for p in water_bplot.patches:
    water_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes.set_title('Percentage of Schools with Water Facility in India',size = 16 , pad = 16, color='white',bbox=dict(facecolor='black', alpha=1.0))

axes.set_ylabel('Percentage',size=13)
axes.set_xlabel('Year',size=13)

water_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

- This plot makes me really happy as there is a consistent increase and nearly 98% of the schools have water facilities for children across India.

In [ ]:
# setting the plot styles and backgrounds
sns.set(style="ticks",context="talk")
plt.style.use('seaborn-deep')

f, axes = plt.subplots(1, 1, figsize=(20, 10))

# barplot for computer facility
comps_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_comps,palette = 'Pastel2',edgecolor='black',ax=axes)
comps_bplot.set(ylim=(0, 80))
for p in comps_bplot.patches:
    comps_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes.set_title('Percentage of Schools with Computer Facility in India',size = 16 , pad = 16, color='white',bbox=dict(facecolor='black', alpha=1.0))

axes.set_ylabel('Percentage',size=13)
axes.set_xlabel('Year',size=13)

comps_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

- In a world where Artificial Intelligence is taking over and technology keeps progressing by the minute, it's not just a want but a need to get the students up to date with new technology by giving them Computer Facilities starting from Primary Schools.
- Although there is a small increase every year, it's considerably low. I hope Indian Government takes this into account.

In [ ]:
# setting the plot styles and backgrounds
sns.set(style="ticks",context="talk")
plt.style.use('seaborn-deep')

f, axes = plt.subplots(1, 1, figsize=(20, 10))

# barplot for electricity facility
elec_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_elec,palette = 'CMRmap',edgecolor='black',ax=axes)
elec_bplot.set(ylim=(0, 125))
for p in elec_bplot.patches:
    elec_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes.set_title('Percentage of Schools with Electricity Facility in India',size = 16 , pad = 16, color='white',bbox=dict(facecolor='black', alpha=1.0))

axes.set_ylabel('Percentage',size=13)
axes.set_xlabel('Year',size=13)

elec_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

- Another important need and I see a consistent rise in Percentage across all Schools.

In [ ]:
# setting the plot styles and backgrounds
sns.set(style="ticks",context="talk")
plt.style.use('seaborn-deep')

# creating two subplots: girls and boys
f, axes = plt.subplots(1, 2, figsize=(20, 10))

# -------------------------------------------------------------- #

# barplot for girls
t_girls_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_gt,palette='rainbow',edgecolor='black',ax=axes[0])
t_girls_bplot.set(ylim=(40, 130))
for p in t_girls_bplot.patches:
    t_girls_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes[0].set_title('Percentage of Toilet Facilities for Girls in India',size = 16 , pad = 16, color='white',bbox=dict(facecolor='black', alpha=1.0))

axes[0].set_ylabel('Percentage',size=13)
axes[0].set_xlabel('Year',size=13)

t_girls_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

# ------------------------------------------------------------------- #

# barplot for boys
t_boys_bplot = sns.barplot(x="Year",y='Value',hue='Variable',data=tidy_bt,palette='rainbow',edgecolor='black',ax=axes[1])
t_boys_bplot.set(ylim=(40, 130))
for p in t_boys_bplot.patches:
    t_boys_bplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', fontsize = 13,
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# highlighting the title
axes[1].set_title('Percentage of Toilet Facilities for Boys in India',size = 16 , pad = 16, color='white',bbox=dict(facecolor='black', alpha=1.0))

axes[1].set_ylabel('Percentage',size=13)
axes[1].set_xlabel('Year',size=13)

t_boys_bplot.legend(fancybox=True, framealpha=1, borderpad=0.5)

- If you notice in 2013-14 school year, the facilities of toilets is really low for Higher Secondary Schools but they have a sudden growth from 2014-2015. 
- Thanks to Indian Government that there are many programs and intiatives undertaken by States and the Centre to ensure people have basic sanitary facilities in every field.

# Conclusion

We come to the end of our **Indian Schools - Actions to be taken by Indian Govt** report. I hope you all liked my presentation and this is just a beginning to me. I also hope this is a good starter notebook for beginners. Please let me know if I made any mistake and if there's any other way to improve my reporting and EDA skills. I will be glad to go over them.

In case you find this useful, please **UPVOTE** and leave a **COMMENT**. This would greatly encourage me to pursue Kaggling!

Thank you and Stay Safe!

LinkedIn: [Vidya](https://www.linkedin.com/in/vidyapb/).